# Volume 3: Discrete Hidden Markov Models
    Daniel Perkins
    MATH 407
    1/20/25

In [167]:
import numpy as np
import string
import codecs
from hmmlearn.hmm import CategoricalHMM

# Problems 1-5
This is the HMM class that you will be adding functions to throughout the lab.

In [157]:
class HMM(object):
    """
    Finite state space hidden Markov model.
    """
    
    # Problem 1
    def __init__(self, A, B, pi):
        """
        Initialize an HMM with parameters A, B, and pi.
        """
        self.A = A
        self.B = B
        self.pi = pi
    
    
    # Problem 2
    def forward_pass(self, z):
        """
        Compute the forward probability matrix.

        Parameters
        ----------
        z : ndarray of shape (T,)
            The observation sequence

        Returns
        -------
        alpha : ndarray of shape (T, n)
            The forward probability matrix
        """
        # Initialize alpha as an empty matrix
        T, n = np.shape(z)[0], np.shape(self.A)[0]
        alpha = np.empty((T, n))
        
        # Compute alpha using array broadcasting (formula on page 268)
        alpha[0] = self.pi * self.B[z[0]]
        for t in range(1, T):
            alpha[t] = self.B[z[t], :] * np.dot(alpha[t - 1], self.A.T)
        
        return alpha
        
    
    # Problem 4
    def backward_pass(self, z):
        """
        Compute the backward probability matrix and gamma values.

        Parameters
        ----------
        z : ndarray of shape (T,)
            The observation sequence

        Returns
        -------
        beta : ndarray of shape (T, n)
            The backward probability matrix
        gamma : ndarray of shape (T, n)
            The state probability matrix
        """
        # Initialize the arrays
        T, n = np.shape(z)[0], np.shape(self.A)[0]
        beta, gamma = np.empty((T, n)), np.empty((T, n))
        
        # Use array broadcasting and matrix multiplication to compute beta
        beta[T-1] = np.ones(n)
        for t in range(T-2, -1, -1):
            beta[t, :] = (self.A.T @ (beta[t+1, :] * self.B[z[t+1], :]))
        
        # Find gamma
        alpha = self.forward_pass(z)
        gamma = (alpha * beta) / np.sum(alpha[-1, :])
        
        return beta, gamma
        
    
    # Problem 5
    def viterbi_algorithm(self, z):
        """
        Compute the most likely hidden state sequence using the Viterbi algorithm.

        Parameters
        ----------
        z : ndarray of shape (T,)
            The observation sequence

        Returns
        -------
        x*: ndarray of shape (T,)
            The most likely state sequence
        """
        T, n = np.shape(z)[0], np.shape(self.A)[0]
        
        # Compute eta using array broadcasting
        eta = np.empty((T, n))
        eta[0] = self.B[z[0]] * self.pi                
        for t in range(1, T):
            eta[t, :] = np.max(self.B[z[t], :, None] * self.A * eta[t-1, None, :], axis=1)
    
        # Compute x using array broadcasting
        x = np.empty_like(z)
        x[T-1] = np.argmax(eta[T-1])            
        for t in range(T-2, -1, -1):
            x[t] = np.argmax(self.A[x[t+1], :] * eta[t, :])
            
        return x

# Problem 2 test case

Use the following HMM and code to test your HMM class.
Compare the output to `forward_pass` with the lab pdf.

In [107]:
pi = np.array([.6, .4])
A = np.array([[.7, .4], [.3, .6]])
B = np.array([[.1, .7], [.4, .2], [.5, .1]])
z_example = np.array([0, 1, 0, 2])
example_hmm = HMM(A, B, pi)

alpha = example_hmm.forward_pass(z_example)
print(np.sum(alpha[-1, :]))

0.009629599999999999


# Problem 3

Consider the following (very simplified) model of the price of a stock over time as an HMM.
The observation states will be the change in the value of the stock.
For simplicity, we will group these into five values: large decrease, small decrease, no change, small increase, large increase, labeled as integers from 0 to 4.
The hidden state will be the overall trends of the market.
We'll consider the market to have three possible states: declining in value (bear market), not changing in value (stagnant), and increasing in value (bull market), labeled as integers from 0 to 2.
Let the HMM modeling this scenario have parameters
$$
\boldsymbol\pi=\begin{bmatrix}
1/3 \\ 1/3 \\ 1/3
\end{bmatrix},
\quad
A=\begin{bmatrix}
0.5 & 0.3 & 0 \\
0.5 & 0.3 & 0.3 \\
0 & 0.4 & 0.7
\end{bmatrix},
\quad
B=\begin{bmatrix}
0.3 & 0.1 & 0 \\
0.3 & 0.2 & 0.1 \\
0.3 & 0.4 & 0.3 \\
0.1 & 0.2 & 0.4 \\
0 & 0.1 & 0.2
\end{bmatrix}
$$
The file `stocks.npy` contains a sequence of 50 observations drawn from this HMM.
What is the probability of this observation sequence given these model parameters?
Use your implementation of the forward pass algorithm from Problem 2 to find the answer.
Note that the answer is very small, because there are lots of possible observation sequences.

In [121]:
# HMM parameter setup
pi = np.array([1/3, 1/3, 1/3])
A = np.array([
    [0.5, 0.3, 0.0],
    [0.5, 0.3, 0.3],
    [0.0, 0.4, 0.7]
])
B = np.array([
    [0.3, 0.1, 0.0],
    [0.3, 0.2, 0.1],
    [0.3, 0.4, 0.3],
    [0.1, 0.2, 0.4],
    [0.0, 0.1, 0.2]
])

In [123]:
# Load in data
data = np.load("stocks.npy")

# Create the HMM and find the probability of the data
hmm = HMM(A, B, pi)
alpha = hmm.forward_pass(data)
probability = np.sum(alpha[-1, :])
print("The probability of this observation, given these parameters is:\n", probability)

The probability of this observation, given these parameters is:
 6.671115114537779e-34


# Problem 4

Create a method `backward_pass` in your HMM class to implement the backward pass algorithm.
This function should accept the observation sequence $\mathbf{z}$ and return two arrays of the $\beta_t(i)$ and $\gamma_t(i)$ values.

Test your function on the example HMM, and compare the output with the lab pdf.

With your function and the stock model from Problem 3, answer the following question: given the observation sequence in `stocks.npy`, what is the most likely initial hidden state $X_0$?

In [119]:
pi = np.array([.6, .4])
A = np.array([[.7, .4], [.3, .6]])
B = np.array([[.1, .7], [.4, .2], [.5, .1]])
z_example = np.array([0, 1, 0, 2])
example_hmm = HMM(A, B, pi)

# Test case; compare your output with what is in the lab pdf
beta, gamma = example_hmm.backward_pass(z_example)
print(beta)
print(gamma)

[[0.0302  0.02792]
 [0.0812  0.1244 ]
 [0.38    0.26   ]
 [1.      1.     ]]
[[0.18816981 0.81183019]
 [0.51943175 0.48056825]
 [0.22887763 0.77112237]
 [0.8039794  0.1960206 ]]


In [130]:
# HMM parameter setup
pi = np.array([1/3, 1/3, 1/3])
A = np.array([
    [0.5, 0.3, 0.0],
    [0.5, 0.3, 0.3],
    [0.0, 0.4, 0.7]
])
B = np.array([
    [0.3, 0.1, 0.0],
    [0.3, 0.2, 0.1],
    [0.3, 0.4, 0.3],
    [0.1, 0.2, 0.4],
    [0.0, 0.1, 0.2]
])

# Load in data
data = np.load("stocks.npy")

# Create the HMM and find the probability of the data
example_hmm = HMM(A, B, pi)
beta, gamma = example_hmm.backward_pass(data)
most_likely_initial_state = np.argmax(gamma[0])

print(f"The most likely initial hidden state X_0 is:\nState {most_likely_initial_state}")

The most likely initial hidden state X_0 is:
State 1


# Problem 5
Creating a method `viterbi_algorithm` in your HMM class to implement the Viterbi algorithm.
This function should accept the observation sequence $\mathbf{z}$ and return the most likely state sequence $\mathbf{x}^*$.

Test your function on the example HMM and compare output with the lab pdf.

Apply your function to the stock market HMM from Problem 3.
With the observaition sequence from `stocks.npy`, what is the most likely sequence of hidden states?
Is the initial state of the most likely sequence the same as the most likely initial state you found in Problem 4?

In [158]:
pi = np.array([.6, .4])
A = np.array([[.7, .4], [.3, .6]])
B = np.array([[.1, .7], [.4, .2], [.5, .1]])
z_example = np.array([0, 1, 0, 2])
example_hmm = HMM(A, B, pi)

# Test case
xstar = example_hmm.viterbi_algorithm(z_example)
print(xstar)

[1 1 1 0]


In [161]:
# HMM parameter setup
pi = np.array([1/3, 1/3, 1/3])
A = np.array([
    [0.5, 0.3, 0.0],
    [0.5, 0.3, 0.3],
    [0.0, 0.4, 0.7]
])
B = np.array([
    [0.3, 0.1, 0.0],
    [0.3, 0.2, 0.1],
    [0.3, 0.4, 0.3],
    [0.1, 0.2, 0.4],
    [0.0, 0.1, 0.2]
])

# Load in data
data = np.load("stocks.npy")

# Create the HMM and find the probability of the data
hmm = HMM(A, B, pi)
xstar = hmm.viterbi_algorithm(data)

print("The most likely sequence of hidden states is:")
print(xstar)

The most likely sequence of hidden states is:
[0 0 0 0 0 1 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 1 0 0 0 1 2 2 2 2 2
 2 2 2 2 2 2 1 0 0 0 0 0 1]


The most likely sequence of hidden states is given above. This time, the initial state is not the same as what we found in problem 4. This makes sense because the Viterbi algorithm relies on the initial state distribution, using more information than problem 4 (as it is measuring a different thing).

# Problem 6

Train a `hmmlearn.hmm.CategoricalHMM` on `declaration.txt`. Use `N=2` states and `M=len(set(obs))=27` observation values (26 lower case characters and 1 whitespace character).
Use `n_iter=200` and `tol=1e-4`.

Once the learning algorithm converges, analyze the state observation matrix $B$. Note which rows correspond to the largest and smallest probability values in each column of $B$, and check the corresponding characters. The HMM should have detected a vowel state and a consonant state.

In [162]:
def vec_translate(a, my_dict):
    # translate numpy array from symbols to state numbers or vice versa
    return np.vectorize(my_dict.__getitem__)(a)

def prep_data(filename):
    """
    Reads in the file and prepares it for use in an HMM.
    Returns:
        symbols (dict): a dictionary that maps characters to their integer values
        obs_sequence (ndarray): an array of integers representing the read-in text
    """
    # Get the data as a single string
    with codecs.open(filename, encoding='utf-8') as f:
        data = f.read().lower()  # and convert to all lower case
    # remove punctuation and newlines
    remove_punct_map = {ord(char): 
                        None for char in string.punctuation+"\n\r"}
    data = data.translate(remove_punct_map)
    # make a list of the symbols in the data
    symbols = sorted(list(set(data)))
    # convert the data to a NumPy array of symbols
    a = np.array(list(data))
    # make a conversion dictionary from symbols to state numbers
    symbols_to_obsstates = {x: i for i, x in enumerate(symbols)}
    # convert the symbols in a to state numbers
    obs_sequence = vec_translate(a, symbols_to_obsstates)
    return symbols, obs_sequence

In [179]:
# Load in the data
symbols, obs = prep_data('declaration.txt')

# Train the model
h = CategoricalHMM(n_components=2, n_iter=200, tol=1e-4)
h.fit(obs.reshape(-1, 1))

pi = h.startprob_
A = h.transmat_.T
B = h.emissionprob_.T

for i in range(len(B)):
    print(u"{}, {:0.4f}, {:0.4f}".format(symbols[i], *B[i,:]))


 , 0.0517, 0.2975
a, 0.0000, 0.1321
b, 0.0225, 0.0000
c, 0.0437, 0.0000
d, 0.0598, 0.0000
e, 0.0000, 0.2379
f, 0.0427, 0.0000
g, 0.0308, 0.0001
h, 0.0828, 0.0000
i, 0.0000, 0.1243
j, 0.0038, 0.0000
k, 0.0030, 0.0004
l, 0.0541, 0.0000
m, 0.0342, 0.0000
n, 0.1146, 0.0000
o, 0.0037, 0.1378
p, 0.0327, 0.0000
q, 0.0014, 0.0000
r, 0.1008, 0.0000
s, 0.1134, 0.0000
t, 0.1517, 0.0001
u, 0.0000, 0.0579
v, 0.0176, 0.0000
w, 0.0230, 0.0000
x, 0.0021, 0.0000
y, 0.0090, 0.0119
z, 0.0009, 0.0000


It appears, that after a few runs, one of them results in a column where only the vowels have high probabilities (and the constants are practically at 0). This makes sense because vowels are the most common letters in the English language.

# Problem 7

Repeat the same calculation with `WarAndPeace.txt` with 2 hidden states. Interpret/explain your results. Which Cyrillic characters appear to be vowels?

In [177]:
symbols, obs = prep_data('WarAndPeace.txt')

# Train the model
h = CategoricalHMM(n_components=2, n_iter=200, tol=1e-4)
h.fit(obs.reshape(-1, 1))

pi = h.startprob_
A = h.transmat_.T
B = h.emissionprob_.T

for i in range(len(B)):
    print(u"{}, {:0.4f}, {:0.4f}".format(symbols[i], *B[i,:]))

 , 0.2146, 0.0877
а, 0.0000, 0.1760
б, 0.0250, 0.0000
в, 0.0655, 0.0000
г, 0.0296, 0.0000
д, 0.0385, 0.0000
е, 0.0180, 0.1427
ж, 0.0140, 0.0000
з, 0.0252, 0.0000
и, 0.0017, 0.1315
й, 0.0149, 0.0000
к, 0.0497, 0.0010
л, 0.0719, 0.0000
м, 0.0381, 0.0000
н, 0.0973, 0.0000
о, 0.0000, 0.2407
п, 0.0346, 0.0062
р, 0.0597, 0.0000
с, 0.0513, 0.0280
т, 0.0780, 0.0000
у, 0.0000, 0.0590
ф, 0.0018, 0.0003
х, 0.0111, 0.0000
ц, 0.0049, 0.0000
ч, 0.0167, 0.0038
ш, 0.0109, 0.0000
щ, 0.0047, 0.0000
ъ, 0.0003, 0.0003
ы, 0.0000, 0.0376
ь, 0.0009, 0.0433
э, 0.0000, 0.0066
ю, 0.0079, 0.0024
я, 0.0128, 0.0328
ё, 0.0000, 0.0001


Assuming the results of this model are conclusive, the letters most likely to be vowels are "a", "e", "и", "o", "y", "ы", "ь", and "я". This is because they have the highest probabilities in the column with a lot of zeroes. Of course, we cannot assume that these are the Russian vowels without more information. But, it does give us a good idea of what is more likely (or at least which letters are the most common in Russian).